In [ ]:
from map_processing.graph_manager import GraphManager
from map_processing.firebase_manager import FirebaseManager
from firebase_admin import credentials

The `GraphManager` class implements the capabilities for this comparison routine (with dependencies to other code in this repository) along with capabilities for result visualization, data caching, and more. In addition to configuring the weight vector that is applied uniformly to all edges, the class also serves as a wrapper for the optimizer configuration of whether vertices connecting odometry to edge vertices use prescaling based on covariance calculations from the tag observations or whether sparse bundle adjustment (SBA) is used for tag vertex observations. The covariance method essentially acts bundle adjustment due to how the covariance matrix is calculated in the InvisibleMapCreator app. Therefore, in both cases, the objective function becomes a function of pixel units rather than Euclidian space. The weight parameters apply differently between these two cases, so the comparison routine is run for each configuration.

Each weight vector is of the form

$$\mathbf{w}_i = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r]^\intercal$$

where:

* Values $a$-$f$ are used for weighting the translation and rotation of odometry nodes.
* Values $g$-$h$ are used for weighting the pixel coordinates of tags and are only used for SBA optimization.
* Values $i$-$l$ are used for weighting the tag observations when covariance-based prescaling is used (i.e., non-SBA optimization).
* Values $m$-$r$ are used for weighting the translation and rotation of dummy nodes ("dummy nodes" being nodes that are used to enforce the optimization to respect the direction of gravity as reported by the phone).

The weight vectors comprising $W$ are included in Section 5.

The following list serves as a reference for mapping the details described in Section 3.2 to code:

* The comparison routine is invoked by invoking the `GraphManager.process_maps` instance method with the `compare` keyword argument as `True`.
* The tag fixing is prescribed by the `fix_tag_vertices` keyword argument to the `as_graph` function.
* The weight vectors $\mathbf{w}_i$ are sourced from the list in the `GraphManager._comparison_graph1_subgraph_weights` class attribute. The default weight vector is prescribed by the `weights_specifier` keyword argument to the `GraphManager` constructor.

For the `optimization_weight` parameter:

* `0` - sensible_default_weights

```python
np.array([
    -6., -6., -6., -6., -6., -6.,
    18, 18, 0, 0, 0, 0,
    0., 0., 0., -1, 1e2, -1
])
```

* `1` - trust_odom

```python
np.array([
    -3., -3., -3., -3., -3., -3.,
    10.6, 10.6, 10.6, 10.6, 10.6, 10.6,
    0., 0., 0., -1, -1, 1e2
]),
```

* `2` - trust_tags

```python
np.array([
    10, 10, 10, 10, 10, 10,
    -10.6, -10.6, -10.6, -10.6, -10.6, -10.6,
    0, 0, 0, -1e2, 3, 3
]),
```

* `3` - new_option

```python
np.array([
    -6., -6., -6., -6., -6., -6.,
    1, 1, 0, 0, 0, 0,
    0., 0., 0., -1, 1e2, -1
])
```

The weights are provided to the graph object (an instance of the `Graph` class) in the `GraphManager._optimize_graph`  instance method; in the `Graph` object, the weights are then stored in the `Edge` objects' information attribute as  diagonal matrices created from weight vectors (see the `Graph.update_edge_information` instance method). Note that the weights are applied as a negated exponent:

$$\text{Edge}_\text{information} = \text{Diag}\left(e^{-\text{weights}}\right)$$

where $\text{Diag}$ represents the `np.diag` function. Therefore, low weight values correspond to high importance, and vice versa.

In [ ]:
optimization_weight = 0

# 0: Sparse bundle adjustment
# 1: Tag prescaling uses the full covariance matrix
# 2: Tag prescaling uses only the covariance matrix diagonal
# 3: Identity prescaling (tag prescaling is a matrix of ones)
prescaling_opt = 0

# If true, then download and cache the unprocessed maps from Firebase
refresh_cache = False

# JSON files in the cache that match this pattern are loaded by the GraphManager
map_pattern = "unprocessed_maps/**/*duncan*2*"

# Show plots if true
visualize = True

# Location of Firebase credentials
credentials_loc = "/home/duncan/.secret/invisible-map-firebase-adminsdk-s924m-de6e08b695.json"

# Set up the graph manager
cred = credentials.Certificate(credentials_loc)

firebase_manager = FirebaseManager(cred)
if refresh_cache:
    firebase_manager.download_all_maps()

graph_manager = GraphManager(optimization_weight, firebase_manager, prescaling_opt)


### Comparative Graph Optimization

#### SBA

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=0, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=0, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=0, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=0, new_weights_specifier=3)

#### Full Covariance Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=1, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=1, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=1, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=1, new_weights_specifier=3)

#### Diagonal Covariance Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=2, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=2, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=2, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=2, new_weights_specifier=3)

#### Identity Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=3, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=3, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=3, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=True, new_pso=3, new_weights_specifier=3)

#### Lone Graph Optimization

*(i.e., non-comparative optimization routine)*

#### SBA

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=0, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=0, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=0, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=0, new_weights_specifier=3)

#### Full Covariance Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=1, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=1, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=1, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=1, new_weights_specifier=3)

#### Diagonal Covariance Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=2, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=2, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=2, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=2, new_weights_specifier=3)

#### Identity Prescaling

##### Default weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=3, new_weights_specifier=0)

##### Prioritizing odometry nodes

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=3, new_weights_specifier=1)

##### Prioritizing tag weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=3, new_weights_specifier=2)

##### New weights

In [ ]:
graph_manager.process_maps(map_pattern, visualize=visualize,
                           compare=False, new_pso=3, new_weights_specifier=3)

